In [18]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns


In [19]:
filepath_enc = 'data/encounter-events.csv'
filepath_loit = 'data/loitering-events.csv'
filepath_tran = 'data/transshipment-vessels.csv'

In [20]:
df_enc = pd.read_csv(filepath_enc)
df_loit = pd.read_csv(filepath_loit)
df_tran = pd.read_csv(filepath_tran)

# encounter-events dataframe metadata

In [21]:
# print(df_enc.head())  # look at first five instance
print(df_enc.describe())  # get some insight in the data distribution
print(df_enc.info())  # get some insight about data quality
print('All columns: ', df_enc.columns)  # print column names
print('Shape: ', df_enc.shape)  # get shape of dataframe

       fishing_vessel_mmsi  transshipment_vessel_mmsi  mean_latitude  \
count         1.168100e+04               1.168100e+04   11681.000000   
mean          3.492475e+08               3.643690e+08      28.148186   
std           8.560155e+07               1.150885e+08      39.851397   
min           2.000080e+06               2.106310e+08     -64.419519   
25%           2.738125e+08               2.733859e+08      -8.003784   
50%           3.382500e+08               3.415370e+08      50.804512   
75%           4.160038e+08               4.124402e+08      58.732077   
max           9.990001e+08               7.600002e+08      78.223238   

       mean_longitude   duration_hr  median_distance_km  median_speed_knots  
count    11681.000000  11681.000000        11681.000000        11681.000000  
mean        33.759551     13.256271            0.059049            0.603595  
std        127.808306     27.669774            0.068599            0.566881  
min       -179.988139      2.000000    

# loitering-events dataframe metadata

In [22]:
print(df_loit.describe())  # get some insight in the data distribution
print(df_loit.info())  # get some insight about data quality
print('All columns: ', df_loit.columns)  # print column names
print('Shape: ', df_loit.shape)  # get shape of dataframe

       transshipment_mmsi  starting_latitude  starting_longitude  \
count        4.656900e+04       46569.000000        46569.000000   
mean         3.988001e+08          13.332827           37.299625   
std          1.222684e+08          35.386898          109.768020   
min          2.000011e+08         -73.196533         -179.998672   
25%          2.738294e+08         -12.631552          -60.474194   
50%          3.578050e+08           8.495400           32.832062   
75%          5.145650e+08          45.397766          150.625595   
max          8.006687e+08          80.176160          179.995407   

       ending_latitude  ending_longitude  median_speed_knots  \
count     46569.000000      46569.000000        46569.000000   
mean         13.380619         37.359351            1.007226   
std          35.322708        109.733637            0.415658   
min         -87.215640       -179.985580            0.000000   
25%         -12.475266        -60.391735            0.748288   
50%

# transshipment-events dataframe metadata

In [23]:
print(df_tran.describe())  # get some insight in the data distribution
print(df_tran.info())  # get some insight about data quality
print('All columns: ', df_tran.columns)  # print column names
print('Shape: ', df_tran.shape)  # get shape of dataframe

               mmsi           imo
count  1.070000e+03  1.074000e+03
mean   3.956385e+08  8.618462e+06
std    1.330097e+08  6.169946e+05
min    5.004000e+03  5.007510e+06
25%    2.738432e+08  8.225723e+06
50%    3.569255e+08  8.807662e+06
75%    5.141800e+08  9.047266e+06
max    9.927244e+08  9.821134e+06
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124 entries, 0 to 1123
Data columns (total 7 columns):
mmsi               1070 non-null float64
shipname           1123 non-null object
callsign           1106 non-null object
flag               1124 non-null object
imo                1074 non-null float64
first_timestamp    1070 non-null object
last_timestamp     1070 non-null object
dtypes: float64(2), object(5)
memory usage: 61.6+ KB
None
All columns:  Index(['mmsi', 'shipname', 'callsign', 'flag', 'imo', 'first_timestamp',
       'last_timestamp'],
      dtype='object')
Shape:  (1124, 7)
